In [350]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras import regularizers
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score,confusion_matrix

In [586]:
df=pd.read_csv('diabetes.csv')

In [587]:
df.Outcome.value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [588]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [589]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [590]:
df.corr()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
Pregnancies,1.000000,0.129459,0.141282,-0.081672,-0.073535,0.017683,-0.033523,0.544341,0.221898
Glucose,0.129459,1.000000,0.152590,0.057328,0.331357,0.221071,0.137337,0.263514,0.466581
BloodPressure,0.141282,0.152590,1.000000,0.207371,0.088933,0.281805,0.041265,0.239528,0.065068
SkinThickness,-0.081672,0.057328,0.207371,1.000000,0.436783,0.392573,0.183928,-0.113970,0.074752
Insulin,-0.073535,0.331357,0.088933,0.436783,1.000000,0.197859,0.185071,-0.042163,0.130548
BMI,0.017683,0.221071,0.281805,0.392573,0.197859,1.000000,0.140647,0.036242,0.292695
DiabetesPedigreeFunction,-0.033523,0.137337,0.041265,0.183928,0.185071,0.140647,1.000000,0.033561,0.173844
Age,0.544341,0.263514,0.239528,-0.113970,-0.042163,0.036242,0.033561,1.000000,0.238356
Outcome,0.221898,0.466581,0.065068,0.074752,0.130548,0.292695,0.173844,0.238356,1.000000


In [614]:
X=df.drop('Outcome',axis=1)

In [615]:
y=df['Outcome']

In [616]:
def outliers(col,):
    
    Q1 = np.percentile(X[col], 25)
    Q3 = np.percentile(X[col], 75)
    IQR = Q3 - Q1
    upper = Q3 + 1.5* IQR
    lower = Q1 - 1.5* IQR
    rem=X[(X[col] > upper) | (X[col] < lower)].index
    X.drop(rem,axis=0,inplace=True)
    y.drop(rem,axis=0,inplace=True)

In [617]:
for i in X.columns:
    outliers(i)

In [618]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

In [619]:
std_sl=StandardScaler()
X_train=std_sl.fit_transform(X_train)
X_test = std_sl.transform(X_test)

In [620]:
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y= y_train)
class_weights_dict = dict(enumerate(class_weights))

In [621]:
n_features=X_train.shape[1]

In [622]:
model=Sequential()

In [623]:
model.add(Dense(32,activation='relu',input_shape=(n_features,)))

In [624]:
model.add(Dense(16,activation='relu'))

In [625]:
model.add(Dropout(.2))

In [626]:
model.add(Dense(8,activation='relu'))

In [627]:
model.add(Dense(1,activation='sigmoid'))

In [628]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [629]:
model.fit(X_train,y_train,batch_size=32,validation_data=(X_test,y_test),epochs=150,class_weight=class_weights_dict)

Epoch 1/150
14/14 [==============================] - 1s 15ms/step - loss: 0.7059 - accuracy: 0.3685 - val_loss: 0.7520 - val_accuracy: 0.3874
Epoch 2/150
14/14 [==============================] - 0s 5ms/step - loss: 0.6736 - accuracy: 0.4270 - val_loss: 0.7079 - val_accuracy: 0.4555
Epoch 3/150
14/14 [==============================] - 0s 5ms/step - loss: 0.6520 - accuracy: 0.5169 - val_loss: 0.6741 - val_accuracy: 0.5759
Epoch 4/150
14/14 [==============================] - 0s 4ms/step - loss: 0.6424 - accuracy: 0.5843 - val_loss: 0.6532 - val_accuracy: 0.6649
Epoch 5/150
14/14 [==============================] - 0s 5ms/step - loss: 0.6212 - accuracy: 0.6090 - val_loss: 0.6315 - val_accuracy: 0.6859
Epoch 6/150
14/14 [==============================] - 0s 4ms/step - loss: 0.6202 - accuracy: 0.6562 - val_loss: 0.6036 - val_accuracy: 0.7277
Epoch 7/150
14/14 [==============================] - 0s 4ms/step - loss: 0.6023 - accuracy: 0.6809 - val_loss: 0.5752 - val_accuracy: 0.7592
Epoch 8/150


Epoch 59/150
14/14 [==============================] - 0s 4ms/step - loss: 0.4352 - accuracy: 0.7910 - val_loss: 0.3924 - val_accuracy: 0.8220
Epoch 60/150
14/14 [==============================] - 0s 4ms/step - loss: 0.4245 - accuracy: 0.8270 - val_loss: 0.3919 - val_accuracy: 0.8272
Epoch 61/150
14/14 [==============================] - 0s 4ms/step - loss: 0.4336 - accuracy: 0.7888 - val_loss: 0.3872 - val_accuracy: 0.8220
Epoch 62/150
14/14 [==============================] - 0s 4ms/step - loss: 0.4172 - accuracy: 0.7865 - val_loss: 0.3823 - val_accuracy: 0.8272
Epoch 63/150
14/14 [==============================] - 0s 4ms/step - loss: 0.4454 - accuracy: 0.8112 - val_loss: 0.3835 - val_accuracy: 0.8325
Epoch 64/150
14/14 [==============================] - 0s 4ms/step - loss: 0.4261 - accuracy: 0.8135 - val_loss: 0.3883 - val_accuracy: 0.8115
Epoch 65/150
14/14 [==============================] - 0s 4ms/step - loss: 0.4213 - accuracy: 0.8045 - val_loss: 0.3815 - val_accuracy: 0.8168
Epoch 

14/14 [==============================] - 0s 4ms/step - loss: 0.3570 - accuracy: 0.8315 - val_loss: 0.3879 - val_accuracy: 0.7906
Epoch 117/150
14/14 [==============================] - 0s 4ms/step - loss: 0.3632 - accuracy: 0.8315 - val_loss: 0.3941 - val_accuracy: 0.7853
Epoch 118/150
14/14 [==============================] - 0s 4ms/step - loss: 0.3454 - accuracy: 0.8404 - val_loss: 0.3916 - val_accuracy: 0.7801
Epoch 119/150
14/14 [==============================] - 0s 4ms/step - loss: 0.3594 - accuracy: 0.8494 - val_loss: 0.3995 - val_accuracy: 0.7853
Epoch 120/150
14/14 [==============================] - 0s 4ms/step - loss: 0.3302 - accuracy: 0.8652 - val_loss: 0.3923 - val_accuracy: 0.7906
Epoch 121/150
14/14 [==============================] - 0s 4ms/step - loss: 0.3349 - accuracy: 0.8629 - val_loss: 0.3994 - val_accuracy: 0.7853
Epoch 122/150
14/14 [==============================] - 0s 4ms/step - loss: 0.3350 - accuracy: 0.8472 - val_loss: 0.3929 - val_accuracy: 0.7801
Epoch 123/150

In [630]:
loss,acc=model.evaluate(X_test,y_test)

6/6 [==============================] - 0s 3ms/step - loss: 0.4091 - accuracy: 0.7696


In [631]:
acc

0.7696335315704346

In [632]:
y_test_pred=model.predict(X_test)
y_test_pred = (y_test_pred >= 0.7)

y_train_pred=model.predict(X_train)
y_train_pred = (y_train_pred >=0.7)

14/14 [==============================] - 0s 1ms/step


In [633]:
accuracy_score(y_train,y_train_pred)

0.903370786516854

In [634]:
accuracy_score(y_test,y_test_pred)

0.8219895287958116

In [635]:
confusion_matrix(y_train,y_train_pred)

array([[291,  16],
       [ 27, 111]], dtype=int64)

In [636]:
confusion_matrix(y_test,y_test_pred)

array([[121,  11],
       [ 23,  36]], dtype=int64)

In [637]:
import pickle

In [639]:
with open('ann_model.pkl','wb') as f:
    pickle.dump(model,f)

In [640]:
loaded_model=pickle.load(open('ann_model.pkl','rb'))

In [641]:
loaded_model.predict(np.array([[6,148,72,35,0,33.6,0.627,50]]))

1/1 [==============================] - 0s 66ms/step


array([[1.]], dtype=float32)